In [ ]:
from os import path
from pprint import pprint
import re, glob

fullSub  = re.compile(r'<(note|fw|num).+?</(note|fw|num)>')
wordSub = re.compile(r'</*w>|<w .*?>')
elemSub  = re.compile(r'</*(div|pb|pc|cb|hi|supplied|unclear|seg|ex).*?/*>')
lbYesSub = re.compile(r'</*lb/*>')
lbNoSub  = re.compile(r'<lb .*?break="no"[^>/]*?/*>')
gabSub   = re.compile(r'<gap.*?/>')
# partSub  = re.compile(r'<ab[^>]+?part[^>]*?>')
ampSub   = re.compile(r'&[^ ]+?;')
spaceSub = re.compile(r'\s*  \s*')
# puntSub = re.compile()

hands = set()

def mss_clean(input_file, output_path):
    outDict = {}
    bodyStart = False
    listWit = False
    metaData = ''
    witnesses = ''
    rawData = ''
#     out = open(output_file, 'w+')
    with open(input_file) as infile:
        file_path = path.splitext(input_file)[0].split('/')
        filename = file_path[-1]
        institution = file_path[-3]
        if len(filename) == 5 and not filename.endswith('S'):
            if filename.startswith('1'):
                filename = 'P' + filename[1:].lstrip('0')
            elif filename.startswith('2'):
                filename = '0' + filename[1:].lstrip('0')
            elif filename.startswith('3'):
                filename = filename[1:].lstrip('0')
            elif filename.startswith('4'):
                filename = 'L' + filename[1:].lstrip('0')
        if institution == 'Birmingham':
            filename = 'B_' + filename
        elif institution == 'Muenster':
            filename = 'M_' + filename
        
        for line in infile.readlines():
            if bodyStart == False:
                if '<body>' in line:
                    witnesses = {s.split('"')[1].rstrip('V') for s in witnesses.split('><') if s.startswith('witness')}
                    hands.update(witnesses)
                    for wit in witnesses:
                        outDict[f'{filename}_{wit}'] = metaData + '<body>'
                    bodyStart = True
                    metaData += line.strip()
                    continue
                if '<listWit>' in line:
                    listWit = True
                    witnesses += line.strip()
                if listWit == True:
                    witnesses += line.strip()
                if '</listWit>' in line:
                    witnesses += line.strip()
                    listWit = False
                metaData += line.strip()
            else:    
#                 pprint(f'lineFirst = {line}')
                line = re.sub(fullSub, '', line)
    #                 pprint(f'lineFirst = {line}')
                line = re.sub(elemSub, '', line)
                line = re.sub(gabSub, ' {gap} ', line)
    #                 pprint(f'lineFirst = {line}')
#                 line = re.sub(partSub, '', line)
    #                 pprint(f'lineFirst = {line}')
                if line.strip().endswith('</w>'):
                    line = line.strip() + ' \n'
                
                # Birmingham transcriptions get different treatment
                if filename.startswith('B_'):
                    line = re.sub(wordSub, ' ', line)
                else:
                    line = re.sub(wordSub, '', line)
                    
                line = re.sub(lbNoSub, '', line)
                line = re.sub(lbYesSub, ' ', line)
                if line.strip().endswith('>'):
                    line = line.strip() + '\n'
                line = line.replace('</ab>', '').replace('\n', '') #.replace('<lb break="no"/>', '').replace('<lb/>', ' ')
                line = re.sub(ampSub, '', line)
#                 line = re.sub(spaceSub, ' ', line)
    #                 pprint(f'lineFinal = {line}\n')
                rawData += line
#         print(witnesses)
        if not witnesses:
#             print('No separate hands found...')
            out = open(f'{output_path}/{filename}.xml', 'w+')
            outData = rawData
            out.write(metaData + outData.replace('</ab>', '')) #.replace('\n', ''))
            out.close()
            
        else:
            for wit in witnesses:
                if wit == '*':
                    wit = 'orig'
#                 print(wit)
                out = open(f'{output_path}/{filename}_{wit}.xml', 'w+')
                if wit == 'orig':
                    wit = '*'
                outData = re.sub(rf'<rdg type="[^"]+" hand="(?!(firsthand|\\{wit})V*)[^"]+"[^>]*?/>|<rdg type="[^"]+" hand="(?!(firsthand|\\{wit})V*)[^"]+?".*?</rdg>', '', rawData)
    #             out.write(outData)
                outData = re.sub(spaceSub, ' ', outData)
                out.write(metaData + outData.replace('</ab>', '')) #.replace('\n', ''))
                out.close()

file_list = glob.glob(path.expanduser('~/github/pthu/sources/manuscripts/mss_transcriptions/**/*.xml'), recursive=True)
# print(file_list)
# pool = Pool()
# pool.map(process_file, file_list)
# pool.close()
# pool.join()
for file in file_list:
    print(f'Converting {file}...\n')
    mss_clean(file, path.expanduser(path.join('~/github/pthu/sources/manuscripts/out/')))

print(hands)
# mss_clean(path.expanduser(path.join('~/github/pthu/sources/manuscripts/mss_transcriptions/Muenster/majuscules/20011.xml')), path.expanduser(path.join('~/github/pthu/sources/manuscripts/test/out')))

In [ ]:
filename = '0111'
if len(filename) < 5 and filename.startswith('0'):
    print('2' + filename.rjust(4, '0'))

In [ ]:
s = set()
s.update({1, 2, 3})